# A practical guidance for Whoosh
### By Ruowei Wang, in 2/1/2018

Whoosh is a fast, featureful full-text indexing and searching library implemented in pure Python with detailed documentation. Programmers can use it to easily add search functionality to their applications and websites. Every part of how Whoosh works can be extended or replaced to meet your needs exactly.

Some of Whoosh's features include:

1.Pythonic API. 

2.Pure-Python and Open source. No compilation or binary packages needed, no mysterious crashes.

3.Fielded indexing and search.

4.Fast indexing and retrieval -- faster than any other pure-Python search solution I know of. See Benchmarks (https://bitbucket.org/mchaput/whoosh/wiki/Benchmarks).

5.Pluggable scoring algorithm (including BM25F), text analysis, storage, posting format, etc.

6.Powerful query language (supporting inexact search and proximity search). 

7.Production-quality pure Python spell-checker (as far as I know, the only one). 
   See http://whoosh.readthedocs.io/en/latest/spelling.html

To install whoosh:
1. if you want to use it in jupyter notebook, you could use the command ``conda install whoosh``
2. ``easy_install Whoosh`` and ``pip install Whoosh`` also works, If you have ``setuptools`` or ``pip`` installed.
3. Download source releases from PyPI at http://pypi.python.org/pypi/Whoosh/. Using ``hg clone http://bitbucket.org/mchaput/whoosh``.

In [1]:
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED,NUMERIC
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer, NgramTokenizer
from whoosh import index
import os, os.path

In [5]:
ngt = NgramTokenizer(minsize=2, maxsize=4)
[token.text for token in ngt(u"suit")]


['su', 'sui', 'suit', 'ui', 'uit', 'it']

Jedes Dokument kann mehrere Felder haben, wie z. B. Titel, Inhalt, Url, Datum usw. Zunächst muss ein Schema für unseren Korpus erstellt werden, um diese Felder von Dokumenten in einem Index zu spezifizieren.

Das Schema ist die Menge aller möglichen Felder in einem Dokument. Jedes einzelne Dokument verwendet möglicherweise nur eine Teilmenge der verfügbaren Felder im Schema.

Anmerkung: Ohne ein Schema kann der Query Parser in Whoosh den Text in der Benutzerabfrage nicht verarbeiten (d. h. er kann keine Phrasensuche durchführen).

Es folgt ein Beispiel für die Erstellung eines Schemas:

In [2]:
schema = Schema(year=NUMERIC(stored=True),
                author=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                title=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                abstract=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                body=TEXT(analyzer=StandardAnalyzer(stoplist=None)),
                subject=KEYWORD(commas=True,scorable=True),
                keywords=KEYWORD(commas=True, scorable=True))

Folgende vordefinierte Feldtypen werde verwendet:

1.whoosh.fields.NUMERIC:
Dieses Feld speichert int-, long- oder Fließkommazahlen in einem kompakten, sortierbaren Format.

2.whoosh.fields.TEXT:
TEXT-Felder können den Text indizieren und Begriffspositionen speichern (standardmäßig ``TEXT(phrase=True)``), um eine Phrasensuche zu ermöglichen.
Dieses Feld verwendet standardmäßig den ``StandardAnalyzer``. Um einen anderen Analyzer zu spezifizieren, wird das Argument ``Analyzer`` im Konstruktor verwendet, z.B. ``TEXT(analyzer=analysis.StemmingAnalyzer())``. 
Die Dokumentation der verschiedenen Analyzer wird hier angezeigt: http://whoosh.readthedocs.io/en/latest/api/analysis.html#analyzers. Der ``StandardAnalyzer`` schreibt die Wörter nur in Kleinbuchstaben und filtert sie mit einer einfachen Stopword-Liste. 
Standardmäßig werden TEXT-Felder nicht gespeichert, was bedeutet, dass der Inhalt dieses Feldes nicht im Suchergebnis angezeigt wird. Normalerweise werden Sie den Textkörper nicht im Suchindex speichern wollen, Sie können jedoch mit TEXT(stored=True) angeben, dass der Text im Index gespeichert werden soll.

3.whoosh.fields.KEYWORD:
Dieser field-type ist für durch Leerzeichen oder Kommata getrennte Schlüsselwörter gedacht. Dieser Typ ist indiziert und durchsuchbar (und optional speicherbar). Er unterstützt keine Phrasensuche.
Um den Wert des Feldes im Index zu speichern, verwenden Sie ``stored=True`` im Konstruktor. Um die Schlüsselwörter automatisch klein zu schreiben, bevor sie indiziert werden, wird ``lowercase=True`` verwendet. Um die Schlüsselwörter durch Kommas zu trennen (um Schlüsselwörter, die Leerzeichen enthalten, zuzulassen), verwenden Sie ``commas=True``, ansonsten werden die Schlüsselwörter durch Leerzeichen getrennt. Um das Schlüsselwortfeld für die Suche zu verwenden, wird ``scorable=True`` verwendet.

Anmerkung: Es gibt viele andere vordefinierte Felder, die der Benutzer auswählen kann, siehe http://whoosh.readthedocs.io/en/latest/api/fields.html#pre-made-field-types.

Anmerkung: Whoosh kann ein Schema auch deklarativ mit der Basisklasse SchemaClass erstellen und die deklarative Klasse an create_in() oder create_index() anstelle einer Schema-Instanz übergeben.

Nachdem das Schema erstellt wurde, wird jedes Dokument im Korpus indiziert. In diesem Beispiel werden nur zwei Bücher "Vom Winde verweht" und "Grimms Märchen" zur Anzeige verwendet. 
Anmerkung:
1. Indizierten Feldern muss ein Unicode-Wert übergeben werden.
2. Das Öffnen eines Writers sperrt den Index zum Schreiben. In einer Multi-Thread- oder Multiprozess-Umgebung kann das Öffnen eines Writers einen Fehler auslösen, wenn bereits ein Writer geöffnet ist. Das erweiterte Writer-Objekt "whoosh.writing.AsyncWriter" und "whoosh.writing.BufferedWriter" kann dieses Problem lösen.

In [3]:
#to create an index in a dictionary
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = index.create_in("indexdir", schema)
#open an existing index object
ix = index.open_dir("indexdir")
#create a writer object to add documents to the index
writer = ix.writer()
#now we can add documents to the index

abstract1=u'''It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson'''

abstract2=u'''Children's and Household Tales (German: Kinder- und Hausmärchen) is a collection of fairy tales first published in 20 December 1812 by the Grimm brothers, Jacob and Wilhelm. The collection is commonly known in English as Grimms' Fairy Tales.'''

writer.add_document(year=u"1936",
                author=u"Margaret Mitchell",
                title=u"Gone with the wind",
                abstract=abstract1,
                subject=u"novel, love",
                keywords=u"Scarlett, Rhett")
writer.add_document(year=u"1812",
                author=u" Jacob and Wilhelm",
                title=u"Grimms' Fairy Tales",
                abstract=abstract2,
                subject=u"story, children",
                keywords=u"The Frog King,  Rapunzel")
#close the writer and save the added documents in the index
#you should call the commit() function once you finish adding the documents otherwise you will cause an error-
#when you try to edit the index next time and open another writer. 
writer.commit()

Nach der Indizierung der Dokumente kann die Abfrage aufgeschrieben und der Abfrage-String durch den QueryParser in ein Abfrageobjekt umgewandelt werden.

Erstellen Sie ein whoosh.qparser.QueryParser-Objekt, übergeben Sie ihm den Namen des zu durchsuchenden Standardfelds und das Schema des zu durchsuchenden Index. 

Der Query-Parser ist auf modularen Plug-ins aufgebaut. Zum Beispiel gibt das ``qparser.WildcardPlugin``, das sich bereits in der Standard-Plugin-Liste des Parsers befindet, dem Parser die Möglichkeit, nach Wildcards zu suchen. Einige häufig verwendete Plug-ins werden im folgenden Code gezeigt.  

Man kann das Plugins Argument beim Erzeugen des Objekts verwenden, um die Standardliste der Plug-ins zu überschreiben, und ``add_plugin()`` und/oder ``remove_plugin_class()`` verwenden, um die im Parser enthaltenen Plug-ins zu ändern. 

Die Liste der verfügbaren Plug-Ins ist: http://whoosh.readthedocs.io/en/latest/api/qparser.html#plug-ins.

WICHTIG.... Der Query-String sollte ein Unicode-Wert sein!

In [12]:
#parsing the query
# this is just a simple parser with default field
parser=QueryParser("abstract",schema=schema) 
#if you want “unfielded” terms to search both the title and content fields,  use a whoosh.qparser.MultifieldParser
#parser = MultifieldParser(["title", "abstract"], schema=schema)
#call parse() on query to parse a query string into a query object
result=parser.parse(u"apple company department")
print(result)

(abstract:apple AND abstract:company AND abstract:department)


In [11]:
#by default, the parser treats the words as if they were connected by AND. 
#Changing the "group" keyword argument if you want it connencted by Or.
# parser = MultifieldParser(["title", "abstract"], schema=schema,group=OrGroup)
parser=QueryParser("abstract",schema=schema, group=OrGroup) 
result=parser.parse(u"apple company department")
print(result)

(abstract:apple OR abstract:company OR abstract:department)


In [14]:
# you can use .add_plugin() to make the parser more powerful
#GtLtPlugin() lets you use >, <, >=, <=, =>, or =< after a field specifier, 
#and translates the expression into the equivalent range:
parser.add_plugin(GtLtPlugin()) 
result=parser.parse(u"year:<2000")
print(result)

year:[ TO 2000}


In [15]:
#FuzzyTermPlugin lets you search for “fuzzy” terms, that is, terms that don’t have to match exactly. 
#The fuzzy term will match any similar term within a certain number of “edits” 
parser.add_plugin(FuzzyTermPlugin())
result=parser.parse(u"author:margare~")#would match a document has Margare and all terms in the index within one “edit” of cat, for example Margaret insert t
print(result)
#searcher object is used for searching the matched documents
#you can open the searcher using a with statement so the searcher is automatically closed when you’re done with it
#ix is the document index we created before
with ix.searcher() as searcher:
    results=searcher.search(result)#The Results object acts like a list of the matched documents.
    print (results[0])

author:margare~
<Hit {'abstract': "It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson", 'author': 'Margaret Mitchell', 'title': 'Gone with the wind', 'year': '1936'}>


In [16]:
#The default phrase query tokenizes the text between the quotes and creates a search for those terms in proximity.
# print parser.default_set()
#use single quotation marks for the unicode string since double quotation marks are used to represent phrases here
result=parser.parse(u'title:"gonE the"~2')# would match a document has wind within 2 words after gone
print(result)

with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)

title:"gone the"
<Top 1 Results for Phrase('title', ['gone', 'the'], slop=2, boost=1.000000) runtime=0.00045420000003559835>


In [17]:
#you can use * or ? for inexact term search
#use ? to represent a single character and * to represent any number of characters
result=parser.parse(u'title:go*')# would match a document has wind within 2 words after gone
print(result)
with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)
    print (results[0])

title:go*
<Top 1 Results for Prefix('title', 'go') runtime=0.00029460000007475173>
<Hit {'abstract': "It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson", 'author': 'Margaret Mitchell', 'title': 'Gone with the wind', 'year': '1936'}>


In [20]:
#If you want to do more complex proximity searches, you can replace the phrase plugin with the whoosh.qparser.SequencePlugin.
#It allows any query between the quotes.

#remove the ability to specify phrase queries inside double quotes.
parser.remove_plugin_class(PhrasePlugin)
#Adds the ability to group arbitrary queries inside double quotes,
#to produce a query matching the individual sub-queries in sequence.
parser.add_plugin(SequencePlugin())
#IMPORTANT!!! Not like phrase query which specify the field outside the double quotation marks,
#you need to specify the field inside the double quotation marks for each subquery
#the query string below represents the query 'abstract:"(child OR childr*) ho*sehold"~3 AND title:tales' 
result=parser.parse(u'"abstract:(child OR childr*) abstract:ho*sehold"~3 AND title:tale*')
print (result)
with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)
#     print (results[0])
    #we can get the position of a term by doing it manually
    import re
    for result in results:
        analyzer=StandardAnalyzer(stoplist=None)
        a=[(t.pos) for t in analyzer(result['abstract'],positions=True) if re.match(r"tale*",t.text)]
        print ("the position of the word pattern "+"<tale*> "+"in document <"+result['title']+"> is:")
        print (a)

(((abstract:child OR abstract:childr*) NEAR abstract:ho*sehold) AND title:tale*)
<Top 1 Results for And([Sequence([Or([Term('abstract', 'child'), Prefix('abstract', 'childr')]), Wildcard('abstract', 'ho*sehold')], slop=3, boost=1.000000), Prefix('title', 'tale')]) runtime=0.0011049000000866727>
the position of the word pattern <tale*> in document <Grimms' Fairy Tales> is:
[4, 14, 38]


### Reference:
Whoosh documentation website. http://whoosh.readthedocs.io/en/latest/index.html